In [36]:
import pandapower as pp
import pandapower.timeseries as ts
import pandas as pd
import numpy as np
from pandapower.control import ConstControl
from pandapower.timeseries import OutputWriter
from pandapower.timeseries import run_timeseries


In [ ]:
import pandapower as pp
from pandapower.plotting import simple_plot
import numpy as np
import plotly

# Create an empty network
net = pp.create_empty_network()

# Create 5 buses (110 kV system)
bus1 = pp.create_bus(net, vn_kv=110, name="Bus 1 (Thermal)")
bus2 = pp.create_bus(net, vn_kv=110, name="Bus 2 (Solar)")
bus3 = pp.create_bus(net, vn_kv=110, name="Bus 3 (Hydro + Load)")
bus4 = pp.create_bus(net, vn_kv=110, name="Bus 4 (Load)")
bus5 = pp.create_bus(net, vn_kv=110, name="Bus 5 (Load)")

# Assign 2D coordinates to buses (pentagon layout for visualization)
r = 100  # Radius for bus placement
angles = np.linspace(0, 2 * np.pi, 5, endpoint=False)
for i, bus in enumerate([bus1, bus2, bus3, bus4, bus5]):
    net.bus.loc[bus, "x"] = r * np.cos(angles[i])
    net.bus.loc[bus, "y"] = r * np.sin(angles[i])

# Create generators with reactive power limits
pp.create_gen(net, bus=bus1, p_mw=100, vm_pu=1.0, slack=True, name="Thermal Gen", min_q_mvar=-50, max_q_mvar=50)
pp.create_gen(net, bus=bus2, p_mw=50, vm_pu=1.0, name="Solar Gen", min_q_mvar=-25, max_q_mvar=25)
pp.create_gen(net, bus=bus3, p_mw=80, vm_pu=1.0, name="Hydro Gen", min_q_mvar=-30, max_q_mvar=30)
0
# Create loads
pp.create_load(net, bus=bus3, p_mw=40, q_mvar=20, name="Load Bus 3")
pp.create_load(net, bus=bus4, p_mw=60, q_mvar=30, name="Load Bus 4")
pp.create_load(net, bus=bus5, p_mw=50, q_mvar=25, name="Load Bus 5")

# Create lines with a 110 kV-compatible standard type
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=10, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 1-2")
pp.create_line(net, from_bus=bus2, to_bus=bus3, length_km=15, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2-3")
pp.create_line(net, from_bus=bus3, to_bus=bus4, length_km=12, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 3-4")
pp.create_line(net, from_bus=bus4, to_bus=bus5, length_km=10, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4-5")
pp.create_line(net, from_bus=bus5, to_bus=bus1, length_km=20, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 5-1")

# Run power flow with increased iterations and diagnostics
try:
    pp.runpp(net, max_iteration=50, tolerance_mva=1e-6)
    print("Power flow converged successfully!")
    print("\nBus Results:")
    print(net.res_bus)
    print("\nGenerator Results:")
    print(net.res_gen)
    print("\nLine Results:")
    print(net.res_line)
except pp.LoadflowNotConverged:
    print("Power flow still did not converge. Try adjusting parameters further.")


numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


Power flow converged successfully!

Bus Results:
      vm_pu  va_degree       p_mw     q_mvar
0  1.000000   0.000000 -21.338364 -16.823845
1  1.000000   0.292583 -50.000000  68.567631
2  1.000000  -0.135297 -40.000000 -31.164615
3  0.990000  -0.410725  60.000000  30.000000
4  0.989426  -0.421299  50.000000  25.000000

Generator Results:
        p_mw     q_mvar  va_degree  vm_pu
0  21.338364  16.823845   0.000000    1.0
1  50.000000 -68.567631   0.292583    1.0
2  80.000000  51.164615  -0.135297    1.0

Line Results:
   p_from_mw  q_from_mvar    p_to_mw  q_to_mvar     pl_mw    ql_mvar  \
0 -25.160830    21.770176  25.301177 -33.197367  0.140347 -11.427191   
1  24.698823   -35.370264 -24.498719  18.219915  0.200104 -17.150349   
2  64.498719    12.944700 -63.947573 -26.167988  0.551145 -13.223288   
3   3.947573    -3.832012  -3.945625  -7.485705  0.001948 -11.317717   
4 -46.054375   -17.514295  46.499194  -4.946331  0.444819 -22.460626   

   i_from_ka   i_to_ka      i_ka  vm_from_pu 

In [21]:
import pandapower as pp
import pandapower.timeseries as ts
import pandas as pd
import numpy as np
from pandapower import runopp
from pandapower.control import ConstControl
from pandapower.timeseries import OutputWriter

# Create the same network as provided
net = pp.create_empty_network()

# Create buses
bus1 = pp.create_bus(net, vn_kv=110, name="Bus 1 (Thermal)")
bus2 = pp.create_bus(net, vn_kv=110, name="Bus 2 (Solar)")
bus3 = pp.create_bus(net, vn_kv=110, name="Bus 3 (Hydro + Load)")
bus4 = pp.create_bus(net, vn_kv=110, name="Bus 4 (Load)")
bus5 = pp.create_bus(net, vn_kv=110, name="Bus 5 (Load)")

# Assign 2D coordinates
r = 100
angles = np.linspace(0, 2 * np.pi, 5, endpoint=False)
for i, bus in enumerate([bus1, bus2, bus3, bus4, bus5]):
    net.bus.loc[bus, "x"] = r * np.cos(angles[i])
    net.bus.loc[bus, "y"] = r * np.sin(angles[i])

# Create generators with cost parameters for OPF
pp.create_gen(net, bus=bus1, p_mw=100, vm_pu=1.0, slack=True, name="Thermal Gen", 
              min_q_mvar=-50, max_q_mvar=50, min_p_mw=0, max_p_mw=150)
pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=50)  # Linear cost for thermal
pp.create_gen(net, bus=bus2, p_mw=50, vm_pu=1.0, name="Solar Gen", 
              min_q_mvar=-25, max_q_mvar=25, min_p_mw=0, max_p_mw=100)
pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)  # Lower cost for solar
pp.create_gen(net, bus=bus3, p_mw=80, vm_pu=1.0, name="Hydro Gen", 
              min_q_mvar=-30, max_q_mvar=30, min_p_mw=0, max_p_mw=120)
pp.create_poly_cost(net, 2, 'gen', cp1_eur_per_mw=30)  # Medium cost for hydro

# Create loads
load3 = pp.create_load(net, bus=bus3, p_mw=40, q_mvar=20, name="Load Bus 3")
load4 = pp.create_load(net, bus=bus4, p_mw=60, q_mvar=30, name="Load Bus 4")
load5 = pp.create_load(net, bus=bus5, p_mw=50, q_mvar=25, name="Load Bus 5")

# Create lines
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=10, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 1-2")
pp.create_line(net, from_bus=bus2, to_bus=bus3, length_km=15, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2-3")
pp.create_line(net, from_bus=bus3, to_bus=bus4, length_km=12, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 3-4")
pp.create_line(net, from_bus=bus4, to_bus=bus5, length_km=10, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4-5")
pp.create_line(net, from_bus=bus5, to_bus=bus1, length_km=20, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 5-1")

# Create sample time series data (24 hours)
n_timesteps = 24
time_steps = range(n_timesteps)

# Sample solar forecast (MW) - varies between 0 and max capacity
solar_forecast = np.sin(np.linspace(0, np.pi, n_timesteps)) * 80  # Sinusoidal pattern
solar_forecast = np.clip(solar_forecast, 0, 100)  # Limit to max capacity

# Sample load forecasts (MW) - some variation around base loads
load3_forecast = 40 + 10 * np.random.normal(1, 0.1, n_timesteps)
load4_forecast = 60 + 15 * np.random.normal(1, 0.1, n_timesteps)
load5_forecast = 50 + 12 * np.random.normal(1, 0.1, n_timesteps)

# Create DataFrames
data = pd.DataFrame({
    'solar_p_mw': solar_forecast,
    'load3_p_mw': load3_forecast,
    'load4_p_mw': load4_forecast,
    'load5_p_mw': load5_forecast,
    'load3_q_mvar': load3_forecast * 0.5,  # Assume constant power factor
    'load4_q_mvar': load4_forecast * 0.5,
    'load5_q_mvar': load5_forecast * 0.5
}, index=time_steps)

# Create data source for time series
ds = ts.DFData(data)

# Create ConstControl for time series
ConstControl(net, element='gen', element_index=1, variable='p_mw', data_source=ds, profile_name='solar_p_mw')
ConstControl(net, element='load', element_index=load3, variable='p_mw', data_source=ds, profile_name='load3_p_mw')
ConstControl(net, element='load', element_index=load4, variable='p_mw', data_source=ds, profile_name='load4_p_mw')
ConstControl(net, element='load', element_index=load5, variable='p_mw', data_source=ds, profile_name='load5_p_mw')
ConstControl(net, element='load', element_index=load3, variable='q_mvar', data_source=ds, profile_name='load3_q_mvar')
ConstControl(net, element='load', element_index=load4, variable='q_mvar', data_source=ds, profile_name='load4_q_mvar')
ConstControl(net, element='load', element_index=load5, variable='q_mvar', data_source=ds, profile_name='load5_q_mvar')

# Create output writer to store results
ow = OutputWriter(net, time_steps, output_path="./results/")
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_bus', 'p_mw')
ow.log_variable('res_gen', 'p_mw')
ow.log_variable('res_gen', 'q_mvar')
ow.log_variable('res_line', 'loading_percent')

# Run time series OPF
try:
    ts.run_time_series(runopp, net, time_steps=time_steps, verbose=True)
    print("Time series OPF completed successfully!")
except Exception as e:
    print(f"Error during OPF: {str(e)}")

# Access results (example)
# bus_results = pd.read_csv("./results/res_bus/vm_pu.csv", index_col=0)
# gen_results = pd.read_csv("./results/res_gen/p_mw.csv", index_col=0)
# print("\nBus Voltage Magnitude (pu):")
# print(bus_results)
# print("\nGenerator Active Power (MW):")
# print(gen_results)

Error during OPF: 'module' object is not callable


In [57]:

# Create the same network as provided
net = pp.create_empty_network()

# Create buses
bus1 = pp.create_bus(net, vn_kv=110, name="Bus 1 (Thermal)")
bus2 = pp.create_bus(net, vn_kv=110, name="Bus 2 (Solar)")
bus3 = pp.create_bus(net, vn_kv=110, name="Bus 3 (Hydro + Load)")
bus4 = pp.create_bus(net, vn_kv=110, name="Bus 4 (Load)")
bus5 = pp.create_bus(net, vn_kv=110, name="Bus 5 (Load)")

# Assign 2D coordinates
r = 100
angles = np.linspace(0, 2 * np.pi, 5, endpoint=False)
for i, bus in enumerate([bus1, bus2, bus3, bus4, bus5]):
    net.bus.loc[bus, "x"] = r * np.cos(angles[i])
    net.bus.loc[bus, "y"] = r * np.sin(angles[i])

# Create generators with cost parameters for OPF
pp.create_gen(net, bus=bus1, p_mw=100, vm_pu=1.0, slack=True, name="Thermal Gen", 
              min_q_mvar=-50, max_q_mvar=50, min_p_mw=0, max_p_mw=150)
pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=50)  # Linear cost for thermal
pp.create_gen(net, bus=bus2, p_mw=50, vm_pu=1.0, name="Solar Gen", 
              min_q_mvar=-25, max_q_mvar=25, min_p_mw=0, max_p_mw=100)
pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)  # Lower cost for solar
pp.create_gen(net, bus=bus3, p_mw=80, vm_pu=1.0, name="Hydro Gen", 
              min_q_mvar=-30, max_q_mvar=30, min_p_mw=0, max_p_mw=120)
pp.create_poly_cost(net, 2, 'gen', cp1_eur_per_mw=30)  # Medium cost for hydro

# Create loads
load3 = pp.create_load(net, bus=bus3, p_mw=40, q_mvar=20, name="Load Bus 3")
load4 = pp.create_load(net, bus=bus4, p_mw=60, q_mvar=30, name="Load Bus 4")
load5 = pp.create_load(net, bus=bus5, p_mw=50, q_mvar=25, name="Load Bus 5")

# Create lines
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=10, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 1-2")
pp.create_line(net, from_bus=bus2, to_bus=bus3, length_km=15, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2-3")
pp.create_line(net, from_bus=bus3, to_bus=bus4, length_km=12, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 3-4")
pp.create_line(net, from_bus=bus4, to_bus=bus5, length_km=10, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4-5")
pp.create_line(net, from_bus=bus5, to_bus=bus1, length_km=20, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 5-1")

# Create sample time series data (24 hours)
n_timesteps = 24
time_steps = range(n_timesteps)

# Sample solar forecast (MW) - varies between 0 and max capacity
solar_forecast = np.sin(np.linspace(0, np.pi, n_timesteps)) * 80  # Sinusoidal pattern
solar_forecast = np.clip(solar_forecast, 0, 100)  # Limit to max capacity

# Sample load forecasts (MW) - some variation around base loads
load3_forecast = 40 + 10 * np.random.normal(1, 0.1, n_timesteps)
load4_forecast = 60 + 15 * np.random.normal(1, 0.1, n_timesteps)
load5_forecast = 50 + 12 * np.random.normal(1, 0.1, n_timesteps)

# Create DataFrame
data = pd.DataFrame({
    'solar_p_mw': solar_forecast,
    'load3_p_mw': load3_forecast,
    'load4_p_mw': load4_forecast,
    'load5_p_mw': load5_forecast,
    'load3_q_mvar': load3_forecast * 0.5,  # Assume constant power factor
    'load4_q_mvar': load4_forecast * 0.5,
    'load5_q_mvar': load5_forecast * 0.5
}, index=time_steps)

# Create data source for time series
ds = ts.DFData(data)

# Create ConstControl for time series
ConstControl(net, element='gen', element_index=1, variable='p_mw', data_source=ds, profile_name='solar_p_mw')
ConstControl(net, element='load', element_index=load3, variable='p_mw', data_source=ds, profile_name='load3_p_mw')
ConstControl(net, element='load', element_index=load4, variable='p_mw', data_source=ds, profile_name='load4_p_mw')
ConstControl(net, element='load', element_index=load5, variable='p_mw', data_source=ds, profile_name='load5_p_mw')
ConstControl(net, element='load', element_index=load3, variable='q_mvar', data_source=ds, profile_name='load3_q_mvar')
ConstControl(net, element='load', element_index=load4, variable='q_mvar', data_source=ds, profile_name='load4_q_mvar')
ConstControl(net, element='load', element_index=load5, variable='q_mvar', data_source=ds, profile_name='load5_q_mvar')

# Create output writer to store results
ow = OutputWriter(net, time_steps, output_path="./results/", output_file_type=".csv")
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_bus', 'p_mw')
ow.log_variable('res_gen', 'p_mw')
ow.log_variable('res_gen', 'q_mvar')
ow.log_variable('res_line', 'loading_percent')

# Run time series OPF
try:
    run_timeseries(net, time_steps=time_steps, verbose=True)
    print("Time series OPF completed successfully!")
except Exception as e:
    print(f"Error during OPF: {str(e)}")

  0%|          | 0/24 [00:00<?, ?it/s]numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
100%|██████████| 24/24 [00:00<00:00, 132.60it/s]

Time series OPF completed successfully!


In [58]:
bus_results = pd.read_csv("results/res_bus/vm_pu.csv", index_col=0, sep=';')
gen_results = pd.read_csv("results/res_gen/p_mw.csv", index_col=0, sep=';')
line_results = pd.read_csv("results/res_line/loading_percent.csv", index_col=0, sep=';')

In [59]:
line_results.head()

,0,1,2,3,4
0,69.637259,71.865920,106.111133,37.811579,97.159819
1,53.042803,75.346556,104.184247,35.236250,93.854765
2,33.466053,75.427636,103.685669,30.991569,89.942548
3,22.063568,82.703034,106.799607,30.773050,91.061585
4,9.524957,81.516933,105.111504,26.060626,85.528432


In [51]:
def run_grid_timeseries(solar_data, load3_data, load4_data, load5_data, n_timesteps=24, output_path="results/"):
    """
    Run a time series OPF simulation for a 5-bus grid with given solar and load data.
    
    Parameters:
    -----------
    solar_data : array-like
        Solar generation forecast in MW (n_timesteps length)
    load3_data : array-like
        Load forecast for bus 3 in MW (n_timesteps length)
    load4_data : array-like
        Load forecast for bus 4 in MW (n_timesteps length)
    load5_data : array-like
        Load forecast for bus 5 in MW (n_timesteps length)
    n_timesteps : int
        Number of time steps for simulation (default: 24)
    output_path : str
        Path to save output results (default: "./results/")
    
    Returns:
    --------
    net : pandapower.auxiliary.AuxiliaryNetwork
        The network object with simulation results
    """
    # Create empty network
    net = pp.create_empty_network()

    # Create buses
    bus1 = pp.create_bus(net, vn_kv=110, name="Bus 1 (Thermal)")
    bus2 = pp.create_bus(net, vn_kv=110, name="Bus 2 (Solar)")
    bus3 = pp.create_bus(net, vn_kv=110, name="Bus 3 (Hydro + Load)")
    bus4 = pp.create_bus(net, vn_kv=110, name="Bus 4 (Load)")
    bus5 = pp.create_bus(net, vn_kv=110, name="Bus 5 (Load)")

    # Assign 2D coordinates
    r = 100
    angles = np.linspace(0, 2 * np.pi, 5, endpoint=False)
    for i, bus in enumerate([bus1, bus2, bus3, bus4, bus5]):
        net.bus.loc[bus, "x"] = r * np.cos(angles[i])
        net.bus.loc[bus, "y"] = r * np.sin(angles[i])

    # Create generators with cost parameters for OPF
    pp.create_gen(net, bus=bus1, p_mw=100, vm_pu=1.0, slack=True, name="Thermal Gen", 
                  min_q_mvar=-50, max_q_mvar=50, min_p_mw=0, max_p_mw=150)
    pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=50)
    pp.create_gen(net, bus=bus2, p_mw=50, vm_pu=1.0, name="Solar Gen", 
                  min_q_mvar=-25, max_q_mvar=25, min_p_mw=0, max_p_mw=100)
    pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)
    pp.create_gen(net, bus=bus3, p_mw=80, vm_pu=1.0, name="Hydro Gen", 
                  min_q_mvar=-30, max_q_mvar=30, min_p_mw=0, max_p_mw=120)
    pp.create_poly_cost(net, 2, 'gen', cp1_eur_per_mw=30)

    # Create loads
    load3 = pp.create_load(net, bus=bus3, p_mw=40, q_mvar=20, name="Load Bus 3")
    load4 = pp.create_load(net, bus=bus4, p_mw=60, q_mvar=30, name="Load Bus 4")
    load5 = pp.create_load(net, bus=bus5, p_mw=50, q_mvar=25, name="Bus 5 (Load)")

    # Create lines
    pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=10, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 1-2")
    pp.create_line(net, from_bus=bus2, to_bus=bus3, length_km=15, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2-3")
    pp.create_line(net, from_bus=bus3, to_bus=bus4, length_km=12, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 3-4")
    pp.create_line(net, from_bus=bus4, to_bus=bus5, length_km=10, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4-5")
    pp.create_line(net, from_bus=bus5, to_bus=bus1, length_km=20, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 5-1")

    # Create time steps
    time_steps = range(n_timesteps)

    # Create DataFrame with input data
    data = pd.DataFrame({
        'solar_p_mw': np.clip(solar_data, 0, 100),  # Limit to max solar capacity
        'load3_p_mw': load3_data,
        'load4_p_mw': load4_data,
        'load5_p_mw': load5_data,
        'load3_q_mvar': load3_data * 0.5,  # Assume constant power factor
        'load4_q_mvar': load4_data * 0.5,
        'load5_q_mvar': load5_data * 0.5
    }, index=time_steps)

    # Create data source for time series
    ds = ts.DFData(data)

    # Create ConstControl for time series
    ConstControl(net, element='gen', element_index=1, variable='p_mw', data_source=ds, profile_name='solar_p_mw')
    ConstControl(net, element='load', element_index=load3, variable='p_mw', data_source=ds, profile_name='load3_p_mw')
    ConstControl(net, element='load', element_index=load4, variable='p_mw', data_source=ds, profile_name='load4_p_mw')
    ConstControl(net, element='load', element_index=load5, variable='p_mw', data_source=ds, profile_name='load5_p_mw')
    ConstControl(net, element='load', element_index=load3, variable='q_mvar', data_source=ds, profile_name='load3_q_mvar')
    ConstControl(net, element='load', element_index=load4, variable='q_mvar', data_source=ds, profile_name='load4_q_mvar')
    ConstControl(net, element='load', element_index=load5, variable='q_mvar', data_source=ds, profile_name='load5_q_mvar')

    # Create output writer
    # ow = OutputWriter(net, time_steps, output_path=output_path, output_file_type=".csv")
    ow = OutputWriter(net, time_steps, output_path=None)
    ow.log_variable('res_bus', 'vm_pu')
    ow.log_variable('res_bus', 'p_mw')
    ow.log_variable('res_gen', 'p_mw')
    ow.log_variable('res_gen', 'q_mvar')
    ow.log_variable('res_line', 'loading_percent')

    # Run time series OPF
    try:
        run_timeseries(net, time_steps=time_steps, verbose=True)
        print("Time series OPF completed successfully!")
    except Exception as e:
        print(f"Error during OPF: {str(e)}")

    # return net
    results = {
        "res_bus_vm_pu": net.res_bus.vm_pu.to_dict(),  # Bus voltages (pu)
        "res_bus_p_mw": net.res_bus.p_mw.to_dict(),    # Bus active power (MW)
        "res_gen_p_mw": net.res_gen.p_mw.to_dict(),    # Generator active power (MW)
        "res_gen_q_mvar": net.res_gen.q_mvar.to_dict(),# Generator reactive power (MVar)
        "res_line_loading_percent": net.res_line.loading_percent.to_dict()  # Line loading (%)
    }
    return results

In [47]:
# Sample solar forecast (MW) - varies between 0 and max capacity
solar_forecast = np.sin(np.linspace(0, np.pi, n_timesteps)) * 80  # Sinusoidal pattern
solar_forecast = np.clip(solar_forecast, 0, 100)  # Limit to max capacity

# Sample load forecasts (MW) - some variation around base loads
load3_forecast = 40 + 10 * np.random.normal(1, 0.1, n_timesteps)
load4_forecast = 60 + 15 * np.random.normal(1, 0.1, n_timesteps)
load5_forecast = 50 + 12 * np.random.normal(1, 0.1, n_timesteps)


In [52]:
run_grid_timeseries(solar_forecast, load3_forecast, load4_forecast, load5_forecast, n_timesteps=24)

  0%|          | 0/24 [00:00<?, ?it/s]numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
100%|██████████| 24/24 [00:00<00:00, 119.30it/s]

Time series OPF completed successfully!


{'res_bus_vm_pu': {0: 1.0,
  1: 1.0,
  2: 0.9999999999999999,
  3: 0.9871637649455528,
  4: 0.9862702372405024},
 'res_bus_p_mw': {0: -108.17878549600255,
  1: -1.4210854715202004e-14,
  2: -30.113287992386276,
  3: 73.53875870653796,
  4: 62.20338259120662},
 'res_gen_p_mw': {0: 108.17878549600255, 1: 9.797174393178826e-15, 2: 80.0},
 'res_gen_q_mvar': {0: -60.139263524277794,
  1: -13.820199003647321,
  2: 92.48634186396397},
 'res_line_loading_percent': {0: 67.7682034815391,
  1: 70.03732988356639,
  2: 103.29969286276297,
  3: 36.81864948953313,
  4: 95.16093510265098}}